In [3]:
import pickle
with open('./datasets/movie_len_hgraph.pkl', 'rb') as f:
    dataset = pickle.load(f)

Using backend: pytorch


In [4]:
dataset

{'train-graph': Graph(num_nodes={'movie': 3706, 'user': 6040},
       num_edges={('movie', 'watched-by', 'user'): 988129, ('user', 'watched', 'movie'): 988129},
       metagraph=[('movie', 'user', 'watched-by'), ('user', 'movie', 'watched')]),
 'val-matrix': <6040x3706 sparse matrix of type '<class 'numpy.int64'>'
 	with 6040 stored elements in COOrdinate format>,
 'test-matrix': <6040x3706 sparse matrix of type '<class 'numpy.int64'>'
 	with 6040 stored elements in COOrdinate format>,
 'item-texts': {'title': array(['Toy Story', 'Jumanji', 'Grumpier Old Men', ..., 'Tigerland',
         'Two Family House', 'Contender, The'], dtype=object)},
 'item-images': None,
 'user-type': 'user',
 'item-type': 'movie',
 'user-to-item-type': 'watched',
 'item-to-user-type': 'watched-by',
 'timestamp-edge-column': 'timestamp'}

In [47]:
import torch as th
from dgl.data.rdf import AIFBDataset

dataset = AIFBDataset()
g = dataset[0]
category = dataset.predict_category
num_classes = dataset.num_classes
test_mask = g.nodes[category].data.pop('test_mask')
test_idx = th.nonzero(test_mask, as_tuple=False).squeeze()
train_mask = g.nodes[category].data.pop('train_mask')
train_idx = th.nonzero(train_mask, as_tuple=False).squeeze()
labels = g.nodes[category].data.pop('labels')
## no val set in rdf datasets, use test set for testing functionality
val_idx = test_idx
# Create csr/coo/csc formats before launching sampling processes
# This avoids creating certain formats in each data loader process, which saves momory and CPU.
g.create_formats_()

entity_features = {entity: th.randn(g.nodes(entity).shape[0], 10) for entity in g.ntypes}

data = train_idx, val_idx, test_idx, 10, labels, num_classes, entity_features, g, category

Done loading data from cached files.


In [8]:
import pygraphviz as pgv
def plot_graph(nxg):
    ag = pgv.AGraph(strict=False, directed=True)
    for u, v, k in nxg.edges(keys=True):
        ag.add_edge(u, v, label=k)
    ag.layout('dot')
    ag.draw('graph.png')

In [43]:
g.nodes[category].data

{'train_mask': tensor([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       dtype=torch.uint8), '_ID': tensor([2407, 2403, 1789, 2415, 2428, 2348, 2434, 2210, 4840, 5013, 5028, 5040,
        5056, 2033, 5079, 1654, 1656, 1657, 2215, 1658, 1659,  779, 2029,  778,

In [44]:
train_mask = g.nodes[category].data.pop('train_mask')
train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze()

In [55]:
labels[26]

tensor(2)

In [58]:
test_idx.shape

torch.Size([36])

In [78]:
import dgl
g = dgl.graph(([0, 1, 2, 3, 4], [1, 2, 3, 4, 0]))
g.ndata['f'] = torch.randn(5,5)

In [84]:
g = dgl.heterograph({
    ('user', 'plays', 'game'): ([0, 1, 1, 2], [0, 0, 2, 1]),
    ('user', 'follows', 'user'): ([0, 1, 1], [1, 2, 2])})

In [85]:
games

Graph(num_nodes={'game': 3, 'user': 3},
      num_edges={('user', 'follows', 'user'): 3, ('user', 'plays', 'game'): 4},
      metagraph=[('user', 'user', 'follows'), ('user', 'game', 'plays')])

In [147]:
g = dgl.heterograph({
    ('user', 'plays', 'game'): ([0, 1, 1, 2], [0, 0, 2, 1]),
    ('game', 'follows', 'user'): ([0, 0, 2, 1], [0, 1, 1, 2])})

In [153]:
g.nodes['user'].data['f'] = torch.rand(3,3)

In [161]:
sg.nodes['user']

NodeSpace(data={'f': tensor([[0.5194, 0.4924, 0.2069]]), '_ID': tensor([1])})

In [160]:
sg = dgl.node_subgraph(g, {'user': [1]})

In [130]:
del g

In [131]:
sg.nodes

Graph(num_nodes={'game': 3, 'user': 3},
      num_edges={('game', 'follows', 'user'): 2, ('user', 'plays', 'game'): 0},
      metagraph=[('game', 'user', 'follows'), ('user', 'game', 'plays')])

In [127]:
sg.edge_ids

<bound method DGLHeteroGraph.edge_ids of Graph(num_nodes={'game': 3, 'user': 3},
      num_edges={('game', 'follows', 'user'): 0, ('user', 'plays', 'game'): 3},
      metagraph=[('game', 'user', 'follows'), ('user', 'game', 'plays')])>

In [111]:
sg

Graph(num_nodes={'game': 3, 'user': 3},
      num_edges={('game', 'follows', 'user'): 0, ('user', 'plays', 'game'): 1},
      metagraph=[('game', 'user', 'follows'), ('user', 'game', 'plays')])

In [96]:
val_num, test_num = 1, 1

In [97]:
train_idx, val_idx, test_idx = l[val_num + test_num:], l[:val_num], l[val_num:val_num + test_num]

In [68]:
import numpy as np
sg = dgl.node_subgraph(g, np.array([4, 1, 0]))

In [69]:
sg.edges()

(tensor([0, 2]), tensor([2, 1]))

In [77]:
sg.ndata[dgl.NID]

tensor([4, 1, 0])

In [72]:
uf

array([[ 0.3850421 ,  0.22674189, -1.84741352,  0.22137193,  1.52183595],
       [-0.95763255, -0.88976565,  0.78320824, -0.31951919, -0.58197541],
       [ 1.23197162,  0.90226999,  0.93500434,  1.51011346, -0.11568565],
       [-0.58628424, -0.91292631,  1.40483205,  0.00529038, -2.54419217]])

In [178]:
import torch.nn.functional as F

def load_subtensor(nfeats, labels, seeds, input_nodes, label_type, is_pad):
    if is_pad:
        batch_inputs = {}
        for k, v in nfeats.items():
            if k is 'user':
                batch_inputs[k] = F.pad(v[input_nodes[k]], (0, 4))
            else:
                batch_inputs[k] = F.pad(v[input_nodes[k]], (84, 0))
    else:
        batch_inputs = {k: v[input_nodes[k]] for k, v in nfeats.items()}
    batch_labels = labels[seeds[label_type]]
    return batch_inputs, batch_labels

In [166]:
user_f = torch.rand(5, 84)

In [168]:
device_f = torch.rand(10, 4)

In [169]:
nfeats = {'user': user_f, 'device': device_f}

In [170]:
input_nodes = {'user': [3, 2], 'device': [2]}
seeds = {'user': [0]}

In [173]:
labels = torch.rand(5)

In [175]:
labels

tensor([0.0044, 0.4891, 0.9182, 0.7458, 0.3784])

In [179]:
print(load_subtensor(nfeats, labels, seeds, input_nodes, 'user', True))

({'user': tensor([[3.3247e-01, 5.6562e-01, 4.1759e-01, 4.8633e-01, 4.8842e-01, 1.4187e-01,
         4.8180e-01, 6.4857e-01, 3.9812e-01, 4.7318e-01, 6.5893e-01, 5.2233e-01,
         9.7249e-01, 9.0417e-01, 8.7274e-01, 6.0203e-01, 9.2536e-04, 5.3350e-01,
         9.6606e-02, 6.8013e-01, 7.0631e-01, 4.1007e-01, 7.7959e-01, 4.4473e-01,
         7.9879e-01, 5.2771e-01, 9.8046e-01, 9.7452e-01, 1.1004e-01, 6.4392e-01,
         7.7666e-01, 3.6995e-02, 7.1267e-02, 1.0936e-01, 5.9152e-01, 1.8647e-01,
         1.0428e-01, 9.2030e-01, 5.6310e-01, 2.8014e-01, 4.8588e-01, 6.5741e-01,
         1.4552e-01, 8.1302e-01, 9.9985e-01, 3.4387e-03, 7.3648e-01, 5.9250e-01,
         2.4703e-01, 8.8805e-01, 1.8916e-02, 9.2397e-01, 1.1219e-01, 7.1373e-01,
         3.7820e-01, 7.4070e-01, 7.8297e-01, 2.4294e-01, 1.5599e-01, 1.2721e-01,
         4.6410e-01, 3.6537e-01, 1.5090e-01, 9.5476e-01, 5.5737e-01, 9.0792e-01,
         5.0263e-01, 4.5643e-02, 8.0699e-01, 2.4443e-01, 4.9102e-01, 3.3647e-01,
         7.0062e-0